In [ ]:
import numpy as np
import pandas as pd
import ipywidgets
import qgrid
import collections

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cloning import *

In [ ]:
rcvenus_seq = "atgagtaaaggcgaagaattgttcactggcgtggtaccgatcctggtagaactggatggcgacgttaatggtcacaagttcagcgttagtggagagggtgaaggtgatgcgacctatggcaaactgaccctgaagctgatctgcacaaccggcaagctgcctgttccttggccgacactggttacaacgctgggctatggcgtacaatgtttcgcacggtacccggaccacatgaagcaacatgacttcttcaagagcgctatgcctgaaggctatgtccaagaaaggactatcttcttcaaagacgacggcaattacaagacacgggccgaagtcaaattcgaaggcgatacgctggtcaacagaatcgagctgaaaggcatcgacttcaaggaagatggcaacatcctgggccataaactggaatataattataacagtcataatgtgtatatcaccgctgacaaacaaaagaatggcatcaaggccaacttcaaaatcagacataacatcgaagatggaggtgttcaactggcagaccactaccaacaaaatactccgatcggcgatggcccggtgctgctgccggataaccattatctgagttatcaaagtaagctgagcaaggatccgaacgaaaaaagagatcatatggttctgctggaattcgtaacggccgcgggcatcacgcatggcatggacgagctgtataaataa"
mvenus_seq = "ATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTGATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTGGGTTATGGTGTTCAATGCTTTGCGAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTATAACTCACACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCGAACTTCAAAATTAGACACAACATTGAAGATGGAGGTGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCTACCAATCTAAGCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAATAA".lower()
mscfp3_seq = "ATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTCTCACTTGGGGTGTTCAATGCTTTGCAAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTACATCTCAGACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCTAACTTCAAAATTAGACACAACATTGAAGATGGAGGCGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTAAGCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAATAA".lower()
mscarlet_seq = "ATGAGTAAAGGAGAAGCTGTGATTAAAGAGTTCATGCGCTTCAAAGTTCACATGGAGGGTTCTATGAACGGTCACGAGTTCGAGATCGAAGGCGAAGGCGAGGGCCGTCCGTATGAAGGCACCCAGACCGCCAAACTGAAAGTGACTAAAGGCGGCCCGCTGCCTTTTTCCTGGGACATCCTGAGCCCGCAATTTATGTACGGTTCTAGGGCGTTCATCAAACACCCAGCGGATATCCCGGACTATTATAAGCAGTCTTTTCCGGAAGGTTTCAAGTGGGAACGCGTAATGAATTTTGAAGATGGTGGTGCCGTGACCGTCACTCAGGACACCTCCCTGGAGGATGGCACCCTGATCTATAAAGTTAAACTGCGTGGTACTAATTTTCCACCTGATGGCCCGGTGATGCAGAAAAAGACGATGGGTTGGGAGGCGTCTACCGAACGCTTGTATCCGGAAGATGGTGTGCTGAAAGGCGACATTAAAATGGCCCTGCGCCTGAAAGATGGCGGCCGCTATCTGGCTGACTTCAAAACCACGTACAAAGCCAAGAAACCTGTGCAGATGCCTGGCGCGTACAATGTGGACCGCAAACTGGACATCACCTCTCATAATGAAGATTATACGGTGGTAGAGCAATATGAGCGCTCCGAGGGTCGTCATTCTACCGGTGGCATGGATGAACTATACAAATAA".lower()

In [ ]:
seq = rcvenus_seq

# Single reversion Darwin pilot

## Adjacent stops

In [ ]:
snps = sorted(stop_codon_snps(seq, synonymous=False), key=lambda x: x["residue"])
consecutive_snps = [
    (p1, p2)
    for p1, p2 in zip(snps[:-1], snps[1:])
    if p1["residue"] == p2["residue"] - 1
]

In [ ]:
[(snp1["nt_mutation"], snp2["nt_mutation"]) for (snp1, snp2) in consecutive_snps]

In [ ]:
def double_mutation_substitution(seq, snp1, snp2):
    nt1 = snp1["nt_mutation"]
    nt2 = snp2["nt_mutation"]
    return (nt1[2] + seq[nt1[0] + 1 : nt2[0]] + nt2[2], nt1[0])


primers = [
    {
        **iva_substitution_primers(seq, *double_mutation_substitution(seq, snp1, snp2)),
        "aa_mutation_name": "_".join([s["aa_mutation_name"] for s in (snp1, snp2)]),
    }
    for (snp1, snp2) in consecutive_snps
]
primers = [p for p in primers if "forward_primer" in p]
primers = pd.DataFrame(primers)
primers[primers.forward_primer.str.len() <= 60]

In [ ]:
primers

### IVA

In [ ]:
jqs_num = 191
prefix = "rcVenus"
ordered_seqs = set()
primers_for_output = defaultdict(list)
for p in primers.itertuples():
    for suffix, seq in (("f", p.forward_primer), ("r", p.reverse_primer)):
        name = "JQS{}_{}_{}_{}".format(jqs_num, prefix, p.aa_mutation_name, suffix)
        order = "{}\t{}".format(name, seq)
        if seq in ordered_seqs:
            continue
        ordered_seqs.add(seq)
        primers_for_output[suffix].append(order)
        jqs_num += 1
print("\n".join(primers_for_output["f"] + primers_for_output["r"]))

### Darwin oligos for stops

## Mutagenic oligos for reversions

Pick two different mutations; use a single Darwin mutagenic oligo to revert the mutation, resulting in a bright FP. Centered, left-aligned, and right-aligned.

In [ ]:
# also order one conservative design in a tube

In [ ]:
muts = [parse_mutation_name(s) for s in ["G91A", "L60H", "E34taa"]]